In [ ]:
import numpy as np
class ScratchKMeans:
  def _init(self, n_clusters, n_init, max_iter, tol, verbose=False):
    self.n_clusters=n_clusters
    self.n_init=n_init
    self.max_iter=max_iter
    self.tol=tol
    self.verbose=verbose

#[Problem 1] Determine the initial value of the center point


#[Problem 2] Creation of a function to obtain SSE

  def calc_sse(self, X):
      sse = 0
      for i in range(len(X)):
          for j in range(self.n_clusters):
              if j == X[i, -1]:
                  sse += np.sum((X[i, :-1] - self.myu[j]) ** 2)
      if self.best_sse > sse:
          self.best_sse = sse
          self.record_myu = self.myu
          self.record_cluster = self.n_clusters

In [ ]:
#[Problem 1] Determine the initial value of the center point

from sklearn.datasets import make_blobs

X, _ = make_blobs(
    n_samples=100,
    n_features=2,
    centers=4,
    cluster_std=0.5,
    shuffle=True,
    random_state=0,
)

self.myu = X[np.random.choice(len(X), size=self.n_clusters, replace=False)]
